In [7]:
import numpy as np
import scipy.linalg as la
from scipy.linalg import expm
from concurrent.futures import ProcessPoolExecutor,ThreadPoolExecutor, as_completed
import multiprocessing
from Qdrift import *
from tqdm import tqdm


In [8]:
#1D Hamiltonian definitions
X = np.array([[0.,1.],[1.,0.]],dtype=complex)
Y = np.array([[0.,-1.j],[1.j,0.]],dtype=complex)
Z = np.array([[1.,0.],[0.,-1.]],dtype=complex)
I = np.array([[1.,0.],[0.,1.]],dtype=complex)


def define1D_Hterm(nsites,site,pauli,twosites=False):
    #calculates matrix of a single Hamiltonian term with spectral norm=1
    #nsites: number of spin 1/2 sites
    #site: site index of the term Hi
    #pauli: can be X Y Z for site i
    #twosites: False=> site i, True=> site i and i+1 
    op_list = [I] * nsites
    if not twosites:
        op_list[site]= pauli
    else:
        op_list[site]= pauli
        op_list[(site+1)%nsites] = pauli

    result = op_list[0]
    for op in op_list[1:]:
        result = np.kron(result, op)
    return result

def define1D_Hamiltonian(nsites,terms,coefficients,d=2):
    #dictionary containing all hamiltonian information and initial state
    Hdict=dict()
    Hdict["nsites"]=nsites
    Hdict["local_dim"]=d
    Hdict["Hilbert_dim"]=d**nsites
    Hdict["terms"]=terms
    Hdict["coefficients"]=np.array(coefficients)
    Hdict["lambda"]=sum(coefficients)
    Hdict["probabilities"]=np.abs(Hdict["coefficients"]/Hdict["lambda"])
    Hdict["weighted_terms"]=[hi*Hi for hi,Hi in zip(coefficients,terms)]
    Hdict["Hamiltonian"]=sum(Hdict["weighted_terms"])
    
    # initial updownupdown state
    assert d==2 # if later we want to do qtrits ?
    binary_str = ''.join(['0' if i % 2 == 0 else '1' for i in range(nsites)])
    state_index = int(binary_str, 2)
    state_vector = np.zeros(2**nsites, dtype=complex)
    state_vector[state_index] = 1
    Hdict["initial_state"]=state_vector
    return Hdict

def Ising1D_Hamiltonian(nsites,J=1,h=1,periodic=False):

    if periodic: 
        p=0 
    else: 
        p=1

    Jterms= [define1D_Hterm(nsites,i,Z,twosites=True) for i in range(nsites-p)]
    hterms= [define1D_Hterm(nsites,i,X) for i in range(nsites)]
    Jcoefs= [-J]*len(Jterms)
    hcoefs= [-h]*len(hterms)

    return define1D_Hamiltonian(nsites,Jterms+hterms,Jcoefs+hcoefs)



In [9]:
# Time evolution


#Exact evolution
def evolve_exact(Hdict,total_time,initial_state=None):
    if initial_state is None:
        initial_state=Hdict["initial_state"]
    H=Hdict["Hamiltonian"]

    return la.expm(-1.j*total_time*H)@initial_state

#First order Trotter evolution
def evolve_Trotter(Hdict,total_time,nsteps,initial_state=None):
    if initial_state is None:
        initial_state=Hdict["initial_state"]
    
    dt=total_time/nsteps
    evolved_state=initial_state.copy()
    gates=[]
    for weighted_Hterm in Hdict["weighted_terms"]:
        gates.append(expm(-1j * dt * weighted_Hterm))
    
    for _ in range(nsteps):
        for gate in gates:
            evolved_state= gate @ evolved_state

    return evolved_state

#Qdrift evolution
# in Qdrift.py for ProcessPoolExecutor


In [10]:
# Compute multiple evolutions

def compute_Qdrift_evolutions(Hdict,total_time_list,nsamples_list,ntrajs,initial_state=None):
    # total times list x nsamples list x trajectories x 2**N
    Qstates=np.zeros((len(total_time_list),len(nsamples_list),ntrajs,Hdict["Hilbert_dim"]),dtype=complex)
    with open("compute_Qdrift_evolutions.log","w") as f:
        with tqdm(total=len(total_time_list)*len(nsamples_list),file=f) as progress_bar:
            with ProcessPoolExecutor(max_workers=multiprocessing.cpu_count()) as executor:
                futures = []
                for total_time_idx, total_time in enumerate(total_time_list):
                    for nsamples_idx, nsamples in enumerate(nsamples_list):
                        #Qstates[total_time_idx,nsamples_idx]=evolve_Qdrift_trajectories(Hdict,total_time,nsamples,ntrajs,initial_state)
                        futures.append(executor.submit(evolve_Qdrift_trajectories,Hdict,total_time,nsamples,ntrajs,initial_state,(total_time_idx,nsamples_idx)))

                for future in as_completed(futures):
                    evolved_trajs,idx=future.result()
                    Qstates[idx[0],idx[1]]=evolved_trajs
                    progress_bar.update(1)
            return Qstates

def compute_Trotter_evolutions(Hdict,total_time_list,nsteps_list,initial_state=None):
    # total times list x nsteps list x 2**N
    Trstates=np.zeros((len(total_time_list),len(nsteps_list),Hdict["Hilbert_dim"]),dtype=complex)
    for total_time_idx,total_time in enumerate(total_time_list):
        for nsteps_idx,nsteps in enumerate(nsteps_list):
            Trstates[total_time_idx,nsteps_idx]=evolve_Trotter(Hdict,total_time,nsteps,initial_state)
    return Trstates

def compute_exact_evolutions(Hdict,total_time_list,initial_state=None):
    exactstates=np.zeros((len(total_time_list),Hdict["Hilbert_dim"]),dtype=complex)
    for total_time_idx,total_time in enumerate(total_time_list):
        exactstates[total_time_idx] = evolve_exact(Hdict,total_time,initial_state)
    return exactstates


In [13]:
# Trace distances 
def assert_valid_density_matrix(rho,eigen=False):
    # Check if the matrix is square
    assert rho.shape[0] == rho.shape[1], "Matrix is not square!"
    
    # Check if the matrix is Hermitian (rho = rho^dagger)
    assert np.allclose(rho, rho.conj().T), "Matrix is not Hermitian!"
    
    # Check if the trace is 1
    assert np.isclose(np.trace(rho), 1), f"Trace is not 1, it's {np.trace(rho)}!"
    
    # Check if the matrix is positive semi-definite (all eigenvalues >= 0)
    if eigen:
        eigenvalues = np.linalg.eigvals(rho)
        assert np.all(eigenvalues >= -1e-10), f"Matrix has negative eigenvalues: {eigenvalues}"
    
    return True

def states_to_densitymatrices(states):
    inshape=states.shape
    Hilbert_dim=inshape[-1]
    outshape=inshape + (Hilbert_dim,)
    densitymatrices=np.zeros(outshape,dtype=states.dtype)

    for idx in np.ndindex(inshape[:-1]):
        rho=np.outer(states[idx],states[idx].conj())
        assert_valid_density_matrix(rho)
        densitymatrices[idx]=rho
    return densitymatrices

def equiv_nsamples_list(Hdict,nsteps_list):
    nterms=len(Hdict["terms"])
    return [nterms* nsteps for nsteps in nsteps_list]

def trace_distance(rhoA,rhoB):
    rho_diff = rhoA - rhoB
    return 0.5*np.sum(np.abs(np.real(la.eigvals(rho_diff))))

def partial_trace(rho, site_index, local_dim=2):
    assert_valid_density_matrix(rho) 
    # Total dimension of the Hilbert space (size of rho)
    total_dim = rho.shape[0]
    
    # Calculate the number of sites (log base local_dim of dim_total)
    num_sites = int(np.log(total_dim) / np.log(local_dim))

    # Create a list of all subsystems to trace out (all except the site_index)
    subsystems = [i for i in range(num_sites) if i != site_index]
    
    # Reshape the density matrix into a multidimensional array
    reshaped_rho = rho.reshape([local_dim] * num_sites * 2)
    
    # Perform the partial trace over the subsystems that need to be traced out
    for idx in subsystems:
        reshaped_rho = reshaped_rho.trace(axis1=idx, axis2=idx + num_sites)

    assert_valid_density_matrix(reshaped_rho) 
    return reshaped_rho

def spin_distance(rho1, rho2, site_index,local_dim=2):
    rho1=partial_trace(rho1,site_index, local_dim)
    rho2=partial_trace(rho2,site_index, local_dim)
    # Calculate expectation values for Pauli matrices for both density matrices
    s1_x = np.real(np.trace(np.dot(rho1, X)))
    s1_y = np.real(np.trace(np.dot(rho1, Y)))
    s1_z = np.real(np.trace(np.dot(rho1, Z)))
    
    s2_x = np.real(np.trace(np.dot(rho2, X)))
    s2_y = np.real(np.trace(np.dot(rho2, Y)))
    s2_z = np.real(np.trace(np.dot(rho2, Z)))
    
    # Compute the Euclidean distance (L2 norm) between the vectors of expectation values
    distance = np.linalg.norm([s1_x - s2_x, s1_y - s2_y, s1_z - s2_z])
    
    return distance





In [12]:
ising1D=Ising1D_Hamiltonian(2)
total_time_list=[0.1,0.5]
nsteps_list=[100,150,200,250,300]      #  total gates used = number of Hterms x nsteps
nsamples_list=equiv_nsamples_list(ising1D,nsteps_list)   #  total gates used = nsamples

Qstates=compute_Qdrift_evolutions(ising1D,total_time_list,nsamples_list,100)
Trstates=compute_Trotter_evolutions(ising1D,total_time_list,nsteps_list)
exactstates=compute_exact_evolutions(ising1D,total_time_list)

Qdensitymatrices=states_to_densitymatrices(Qstates)
Trdensitymatrices=states_to_densitymatrices(Trstates)
exactdensitymatrices=states_to_densitymatrices(exactstates)

In [18]:
dm=exactdensitymatrices[0]

partial_trace(dm,0)


(4, 4)